<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [10]:
#!pip install psycopg2
#!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org --trusted-host pypi.python.org psycopg2
import pandas as pd
import psycopg2

In [11]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [96]:
query_3_1 = f'''
SELECT *    
  FROM vacancies
'''

In [97]:
vacancy_df = pd.read_sql_query(query_3_1, connection)
print('Количество вакансий -', vacancy_df['id'].nunique())

Количество вакансий - 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [98]:
query_3_2 = f'''
SELECT *    
  FROM employers
'''

In [99]:
employers_df = pd.read_sql_query(query_3_2, connection)
print('Количество работодателей -', employers_df['id'].nunique())

Количество работодателей - 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [100]:
query_3_3 = f'''
SELECT *    
  FROM areas
'''

In [101]:
areas_df = pd.read_sql_query(query_3_3, connection)
print('Количество регионов -', areas_df['id'].nunique())

Количество регионов - 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [102]:
query_3_4 = f'''
SELECT *    
  FROM industries
'''

In [103]:
industry_df = pd.read_sql_query(query_3_4, connection)
print('Количество сфер деятельности -', industry_df['id'].nunique())

Количество сфер деятельности - 294


***

Из представленной выше статистики можно сделать следующие выводы:
* База данных содержит обширный набор данных, который включает в себя:
    * 49197 уникальных вакансий от 23501 уникальных работодателей.
    * Вышеперечисленные вакансии представленны в 294-х различных сферах деятельности и расположены в 1362-х регионах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [104]:
query_3_5 = f'''
SELECT 
COUNT(v.id) cnt,
a.name  area   
  FROM vacancies v
  JOIN areas a ON v.area_id = a.id
  GROUP BY a.id 
ORDER BY  cnt DESC
LIMIT 5
'''

In [105]:
query_3_5_df = pd.read_sql_query(query_3_5, connection)
print('Пятёрку лидеров по количеству вакансий:\n',  query_3_5_df)

Пятёрку лидеров по количеству вакансий:
     cnt             area
0  5333           Москва
1  2851  Санкт-Петербург
2  2112            Минск
3  2006      Новосибирск
4  1892           Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [106]:
query_3_6 = f'''
SELECT 
id,
salary_from,
salary_to  
  FROM vacancies v
   WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [107]:
query_3_6_df = pd.read_sql_query(query_3_6, connection)
print('Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой -',  query_3_6_df.shape[0])

Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой - 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [108]:
query_3_7 = f'''
SELECT 
AVG(salary_from) salary_bottom,
AVG(salary_to) salary_top
  FROM vacancies v
'''

In [109]:
query_3_7_df = pd.read_sql_query(query_3_7, connection)
print('Среднее значение для нижней границы зарплаты -',  round(query_3_7_df['salary_bottom'][0], 0))
print('Среднее значение для верхней границы зарплаты -',  round(query_3_7_df['salary_top'][0], 0))

Среднее значение для нижней границы зарплаты - 71065.0
Среднее значение для верхней границы зарплаты - 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [110]:
query_3_8 = f'''
SELECT 
COUNT(*) num_vacancies,
employment, 
schedule
  FROM vacancies v
  GROUP BY employment, schedule
  ORDER BY num_vacancies DESC
'''

In [111]:
query_3_8_df = pd.read_sql_query(query_3_8, connection)
query_3_8_df[query_3_8_df['num_vacancies']==query_3_8_df['num_vacancies'].nlargest(2)[1]][['schedule','employment']]

,schedule,employment
1,Удаленная работа,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [112]:
query_3_9 = f'''
SELECT 
COUNT(*) num_vacancies,
experience
  FROM vacancies v
  GROUP BY experience
  ORDER BY num_vacancies ASC
'''

In [113]:
query_3_9_df = pd.read_sql_query(query_3_9, connection)
print('Требуемый опыт работы в порядке возрастания: \n',  query_3_9_df['experience'].iloc[:4])


Требуемый опыт работы в порядке возрастания: 
 0           Более 6 лет
1             Нет опыта
2         От 3 до 6 лет
3    От 1 года до 3 лет
Name: experience, dtype: object


***

Из представленной выше статистики можно сделать следующие выводы:
* Количество вакансий в крупных населенных пунктах имеет положительную ассоциацию с количеством населения в городах (вполне логично):
* Около половины всех вакансий указывают хотя бы один из признаков зарплаты (верхний или нижний предел).
* Касательно предела зарплат, то средняя величина верхней границы заплаты (110537) в полтора раза больше нижней границы заплаты (71065).
* Наиболее популярным сочетанием типа рабочего графика и типа трудоустройства является - "Полный день" и "Полная занятость". Вторым по популярности является "Удаленная работа" и "Полная занятость". Наименее популярной является "Сменный график" и "Проектная работа".
* Наиболее популярным опытом работы исходя из количества вакансий является "От 1 года до 3 лет". Тогда как наименее популярным является "Более 6 лет".

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [114]:
query_3_10 = f'''
SELECT 
COUNT(v.id) num_vacancies,
e.name
  FROM employers e
  JOIN vacancies v ON e.id=v.employer_id
  GROUP BY e.id
  ORDER BY num_vacancies DESC
  LIMIT 5
'''

In [115]:
query_3_10_df = pd.read_sql_query(query_3_10, connection)
print('Работодатель, который находится на первом месте по количеству вакансий -', query_3_10_df['name'][0] )
print('Работодатель, который находится на пятом месте по количеству вакансий -', query_3_10_df['name'][4])

Работодатель, который находится на первом месте по количеству вакансий - Яндекс
Работодатель, который находится на пятом месте по количеству вакансий - Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [116]:
query_3_11 = f'''
SELECT 
a.name region,
COUNT(e.id) num_employers,
COUNT(v.id) num_vacancies
  FROM areas a
  LEFT JOIN employers e ON e.area=a.id
  LEFT JOIN vacancies v ON a.id=v.area_id
  GROUP BY a.id
'''

In [117]:
query_3_11_df = pd.read_sql_query(query_3_11, connection)
no_vacan_index= query_3_11_df[query_3_11_df['num_vacancies']==0]['num_employers'].nlargest(1).index
print('Среди регионов, в которых нет вакансий, наибольшее количество работодателей находится в регионе под названием -', query_3_11_df['region'].iloc[no_vacan_index].iloc[0])

Среди регионов, в которых нет вакансий, наибольшее количество работодателей находится в регионе под названием - Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [118]:
query_3_12 = f'''
SELECT
COUNT(DISTINCT v.area_id) num_regions
  FROM vacancies v
  WHERE employer_id IS NOT NULL and area_id IS NOT NULL
  GROUP BY employer_id
  ORDER BY num_regions DESC
'''

In [120]:
query_3_12_df = pd.read_sql_query(query_3_12, connection)
print('Махимальное количество регионов, в которых работодатель публикует свои вакансии -', query_3_12_df['num_regions'].max())

#Также можно использовать код ниже (т.к. мы уже отсортировали по количеству регионов в запросе)
#print('Махимальное количество регионов, к которых работодатель публикует свои вакансии -', query_3_12_df['num_regions'].iloc[0])

Махимальное количество регионов, в которых работодатель публикует свои вакансии - 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [121]:
query_3_13 = f''' 
SELECT 
COUNT(DISTINCT e.id) employer_num
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  WHERE industry_id IS NULL
  GROUP BY employer_id
  ORDER BY employer_num DESC
'''

In [122]:
query_3_13_df = pd.read_sql_query(query_3_13, connection)
print('Количества работодателей, у которых не указана сфера деятельности -', query_3_13_df['employer_num'].iloc[0])

Количества работодателей, у которых не указана сфера деятельности - 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [123]:
query_3_14 = f''' 
SELECT 
e.name company_name
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  WHERE industry_id IS NOT NULL
  GROUP BY e.id
  HAVING COUNT(e.id)=4
  ORDER BY company_name ASC
  OFFSET 2 LIMIT 1
''' 

In [124]:
query_3_14 = pd.read_sql_query(query_3_14, connection)
print('Название компании находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности -', query_3_14['company_name'].iloc[0])

Название компании находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности - 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [125]:
query_3_15 = f''' 
SELECT 
COUNT(*) employers_num
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  LEFT JOIN industries i ON i.id=e_i.industry_id
  WHERE i.name = 'Разработка программного обеспечения'
'''  

In [126]:
query_3_15 = pd.read_sql_query(query_3_15, connection)
print('Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения» -', query_3_15['employers_num'].iloc[0])

Количество работодателей, у которых в качестве сферы деятельности указана «Разработка программного обеспечения» - 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [127]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import re

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
string_of_cities = page.find('table', class_='standard').text
clean_strings = re.findall('[А-Яа-я-\s]+', string_of_cities)
clean_strings = [ele for ele in clean_strings if ele != ' '] # Удаляем элементы, которые равны пустым значениям
#Создаем кортеж из городов миллионников, чтобы использовать его в SQL запросе
city_tuple = tuple(clean_strings[11:]) #Отбрасываем первый 11 значений в clean_strings, т.к. это текст не относится к городам

In [128]:
query_3_16 = f''' 
SELECT 
a.name,
COUNT(v.id) cnt
  FROM vacancies v 
  JOIN employers e ON e.id=v.employer_id
  JOIN areas a ON a.id=v.area_id
  WHERE e.name = 'Яндекс' AND a.name IN {city_tuple}
GROUP BY a.name  

UNION ALL
SELECT
'Total', COUNT(v.id)
  FROM vacancies v 
  JOIN employers e ON e.id=v.employer_id
  JOIN areas a ON a.id=v.area_id
  WHERE e.name = 'Яндекс' AND a.name IN {city_tuple}
GROUP BY e.name
ORDER BY cnt 
'''

In [129]:
query_3_16_df = pd.read_sql_query(query_3_16, connection)
#print("Фильная таблица: \n", query_3_16)
print("Финальная таблица: \n", query_3_16_df)
print('\n')
print("Количество строк в таблице -", query_3_16_df.shape[0])
print("Значение строки `Total' -", query_3_16_df[query_3_16_df['name'] == 'Total']['cnt'].iloc[0])


Финальная таблица: 
                name  cnt
0              Омск   21
1         Челябинск   22
2        Красноярск   23
3         Волгоград   24
4             Пермь   25
5            Казань   25
6    Ростов-на-Дону   25
7               Уфа   26
8            Самара   26
9         Краснодар   30
10          Воронеж   32
11      Новосибирск   35
12  Нижний Новгород   36
13     Екатеринбург   39
14  Санкт-Петербург   42
15           Москва   54
16            Total  485


Количество строк в таблице - 17
Значение строки `Total' - 485


***

Из представленной выше статистики можно сделать следующие выводы:
* Топ-5 работодателей (по количеству предлагаемых работ) - Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть. На них суммарно приходится 3627 вакансии или 7.4%. 
* Топ-5 регионов по количеству вакансий – Москва, Санкт-Петербург, Минск, Алматы, Новосибирск (код: Приложение-1). 
* Наибольшее количество вакансий, которые публикуют регионы, равно – 181.
* Общее число работодателей, у которых не указана сфера деятельности равна – 8419.
* Топ-5 компаний с наибольшим числом сфер деятельности (5) - Группа компаний МегаМейд, КЗ ПромКотлоСнаб, Транзит, Фирма Гера,  Этажи, федеральная компания (код: Приложение-2).
* Разработкой программного обеспечения занимаются 3553 работодателей. 
* Компания Яндекс предлагает вакансии во всех городах-миллионников (г-м), с наибольшим количеством вакансий в Москве (54) и общим количеством вакансий в г-м равным 485.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [130]:
query_3_17 = f''' 
SELECT 
COUNT(v.id) data_vacancy
  FROM vacancies v
  WHERE lower(v.name) LIKE '%data%' 
  OR lower(v.name) LIKE '%данн%'  
'''

In [131]:
query_3_17_df = pd.read_sql_query(query_3_17, connection)
print("Количество вакансий, которые имеют отношение к данным -", query_3_17_df['data_vacancy'].iloc[0])

Количество вакансий, которые имеют отношение к данным - 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [132]:
query_3_18 = f''' 
SELECT 
COUNT(*) junior_vacancy
  FROM vacancies v
  WHERE (
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
        )
  AND (
    lower (v.name) LIKE '%junior%'
    OR lower (v.experience) LIKE '%нет опыта%'
    OR lower (v.employment) LIKE '%стажировка%'
	  )   
'''

In [133]:
query_3_18_df = pd.read_sql_query(query_3_18, connection)
print("Количество вакансий для дата-сайентиста -", query_3_18_df['junior_vacancy'].iloc[0])

Количество вакансий для дата-сайентиста - 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [135]:
query_3_19 = f''' 
SELECT 
COUNT(*) sql_postgres_skill
  FROM vacancies v
  WHERE (
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
        )
  AND (
    v.key_skills LIKE '%SQL%'
    OR lower(v.key_skills) LIKE '%postgres%'
	  )
'''   	   

In [136]:
query_3_19_df = pd.read_sql_query(query_3_19, connection)
print("Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres -", query_3_19_df['sql_postgres_skill'].iloc[0])

Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres - 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [137]:
query_3_20 = f''' 
SELECT 
COUNT(*) python_skill
  FROM vacancies v
  WHERE (
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
        )
  AND (
    lower(v.key_skills) LIKE '%python%'
	  )	  
'''	  

In [138]:
query_3_20_df = pd.read_sql_query(query_3_20, connection)
print("Количество вакансий для DS, в которых в качестве ключевого навыка указан Python -", query_3_20_df['python_skill'].iloc[0])

Количество вакансий для DS, в которых в качестве ключевого навыка указан Python - 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [139]:
query_3_21 = f'''
SELECT 
--Считаем длину символов в 'навык' и отнимаем длину символов в 'навык' без учета разделителя наываков - CHR(9)
AVG(LENGTH(key_skills) - LENGTH(replace(key_skills, CHR(9), '')) + 1) avg_skill_num
  FROM vacancies v
  WHERE 
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
'''  

In [140]:
query_3_21_df = pd.read_sql_query(query_3_21, connection)
print("Среднее количество ключевых навыков, которое указывают в вакансиях для DS -", round(query_3_21_df['avg_skill_num'].iloc[0],2))

Среднее количество ключевых навыков, которое указывают в вакансиях для DS - 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [141]:
query_3_22 = f'''
SELECT 
v.experience,
AVG(COALESCE((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to, 0)) salary
  FROM vacancies v
  WHERE ( 
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
  )
  AND (v.salary_from IS NOT null
  OR v.salary_to IS NOT NULL)
  GROUP BY v.experience
'''  

In [142]:
query_3_22_df = pd.read_sql_query(query_3_22, connection)
print("Средняя зарплата дата-сайентиста с опытом работы от 3 до 6 лет -", round(query_3_22_df[query_3_22_df['experience'] == 'От 3 до 6 лет']['salary'].iloc[0],0))

Средняя зарплата дата-сайентиста с опытом работы от 3 до 6 лет - 243115.0


***

Из представленной выше статистики можно сделать следующие выводы:
* Процент вакансий имеющих отношение к данным от общего числа вакансий равен 3.6% (код: Приложение-3). 
* Процент вакансий для специалистов уровня Junior от общего числа вакансий имеющих отношение к данным равен 2.9% (код: Приложение-4).
* Процент вакансий для дата-сайентистов где ключевой навык - Python, от общего числа вакансий для дата-сайентистов равен 73.1%. (код: Приложение-5).
* В среднем, количество ключевых навыков в вакансиях для DS равно 6.
* Распределение средней зарплаты дата-сайентиста относительно опыта работы такова (код: Приложение-6):
 
    Нет опыта|	74643.0
    ---------------|---------------
	От 1 года до 3 лет|	139675.0
	От 3 до 6 лет|	243115.0

# Общий вывод по проекту

Исходя из вышепредставленного анализа, можно сделать следующие обобщающие выводы:
* Количество вакансий имеет положительную зависимость от размера города (то есть, размер населения). 
* Пятёрка регионов лидеров по размеру средней зарплаты (код: Приложение-7):

    Регион    | Зарплата
    ---------------|---------------
    Германия | 452751.00
    Болгария | 324348.00
    Испания | 300000.00
    Черногория|  288589.25
     Литва  |263018.50


* Топ пять сфер деятельности по размеру средней зарплаты (код: Приложение-8):

Сфера деятельности|    Зарплата
 ---------------|---------------
Агрохимия (продвижение, оптовая торговля)|  381429.0
Агрохимия (производство) | 339875.0
Игорный бизнес|  191905.0
Корма для животных (производство) | 168300.0
Зоомагазин | 167613.0

* Топ пять сфер деятельности по наименьшей средней зарплате (код: Приложение-8):

Сфера деятельности  |    Зарплата
---------------|---------------    
Детское питание (производство)|  32868.0
Переработка газа|  34068.0
Архив, библиотека, искусствоведение|  36858.0
Ритуальные услуги|  37401.0
Фото- и видеоуслуги | 40961.0

* Топ работодателей по размеру средней зарплате (код: Приложение-9):

Работодатель  |    Зарплата
---------------|---------------
Alpha Personnel | 1000000.0
ЦТП      | 650000.0
RuWork    |   555261.0
Петухова Алия |  552700.0
BeRecruited |  550000.0

``Вывод по зарплате:`` Как видно из описательной статистики, зарплата варьируется от опыта работы, региона, сферы деятельности и работодателя.

* Топ 5 работодателей по количеству вакансий (код: Приложение-10):

Работодатель | # Вакансии
-------------|-------------
Яндекс     |    1933
Ростелеком   |       491
Тинькофф     |     444
СБЕР       |   428
Газпром нефть    |      331

* Топ 5 сфер деятельности по количеству вакансий (код: Приложение-11):

Сфера деятельности | # Вакансии
-------------|-------------
Разработка программного обеспечения     |     12499
Системная интеграция,  автоматизации технологи...    |      11034
Интернет-компания (поисковики, платежные систе...   |        6413
Банк       |    2742
Мобильная связь    |       1585

``Вывод по вакансиям:`` Как видно из описательной статистики, наиболее востребованными считаются специальности связанные с финансовой сферой и интернет/компьютерными технологиями.

``Общее:`` Анализ подобный выше-изложенному можно проводить по множеству разных параметров. 1ое - было бы интересно просмотреть рынок труда в динамике (во времени). Как менялись зарплаты? Какие отрасли были/есть наиболее привлекательны с точки зрения зарплат и количества вакансий? и т.д. 2ое - Исходя из целей исследования, можно было бы посмотреть статистику по отдельным специяльностям. К примеру, посмотерть в каких регионах наиболее востребованны те или иные специалисти; в каких городах та или иная профессия приносит наибольший доход и тд. 3ье - соотнести макроэкономические показатели (пример - ввп, инфляция) с динамикой и характеристикой вакансий. 

### Далее представлен дополнительный код, который использовался для доп. выводов:

* Приложение-1

In [143]:
query_3_11_df['region'].iloc[query_3_11_df['num_vacancies'].nlargest(5).index]

1189             Москва
134     Санкт-Петербург
1208              Минск
71               Алматы
742         Новосибирск
Name: region, dtype: object

* Приложение-2

In [144]:
query_3_14_Extra = f''' 
SELECT 
e.name company_name,
COUNT(e_i.industry_id) num_industries
  FROM employers e
  LEFT JOIN employers_industries e_i ON e_i.employer_id=e.id
  WHERE industry_id IS NOT NULL
  GROUP BY e.id
  ORDER BY num_industries DESC
''' 

query_3_14_extra = pd.read_sql_query(query_3_14_Extra, connection)
query_3_14_extra.head(5)

,company_name,num_industries
0,Группа компаний МегаМейд,5
1,КЗ ПромКотлоСнаб,5
2,Транзит,5
3,Фирма Гера,5
4,"Этажи, федеральная компания",5


* Приложение-3

In [145]:
query_3_17_extra = f''' 
SELECT 
'all_vacancies' type_vacan,
COUNT(id) all_vacancies
FROM vacancies

UNION ALL 
SELECT 
'DS_vacancies' type_vacan,
COUNT(v.id) data_vacancy
  FROM vacancies v
  WHERE lower(v.name) LIKE '%data%' 
  OR lower(v.name) LIKE '%данн%' 
'''
query_3_17_extra_df= pd.read_sql_query(query_3_17_extra, connection)

print('Процент вакансий имеющих отношение к данным от общего числа вакансий равен '
      + str(round(100 * int(query_3_17_extra_df[query_3_17_extra_df['type_vacan']=='DS_vacancies']['all_vacancies'].iloc[0])
                  /int(query_3_17_extra_df[query_3_17_extra_df['type_vacan']=='all_vacancies']['all_vacancies'].iloc[0]),1))+'%.')

Процент вакансий имеющих отношение к данным от общего числа вакансий равен 3.6%.


* Приложение-4

In [146]:
query_3_18_extra = f''' 
SELECT 
'DS_vacancies' type_vacan,
COUNT(v.id) data_vacancy
  FROM vacancies v
  WHERE lower(v.name) LIKE '%data%' 
  OR lower(v.name) LIKE '%данн%' 
  
UNION ALL  
SELECT
'Junior_DS' type_vacan, 
COUNT(*) junior_vacancy
  FROM vacancies v
  WHERE (
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
        )
  AND (
    lower (v.name) LIKE '%junior%'
    OR lower (v.experience) LIKE '%нет опыта%'
    OR lower (v.employment) LIKE '%стажировка%'
	  )   
''' 

query_3_18_extra_df= pd.read_sql_query(query_3_18_extra, connection)

print('Процент вакансий имеющих отношение к данным от общего числа вакансий равен '
      + str(round(100 * int(query_3_18_extra_df[query_3_18_extra_df['type_vacan']=='Junior_DS']['data_vacancy'].iloc[0])
                  /int(query_3_18_extra_df[query_3_18_extra_df['type_vacan']=='DS_vacancies']['data_vacancy'].iloc[0]),1))+'%.')

Процент вакансий имеющих отношение к данным от общего числа вакансий равен 2.9%.


* Приложение-5

In [147]:
query_3_20_extra = f''' 
SELECT
'DS_vacancy' Skills, 
COUNT(*) DS_vacancy_num
  FROM vacancies v
  WHERE (
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
        )

UNION ALL
SELECT 
'DS_Python' Skills,
COUNT(*) python_skill
  FROM vacancies v
  WHERE (
  lower(v.name) LIKE '%data scientist%' 
  OR lower(v.name) LIKE '%data science%'
  OR lower(v.name) LIKE '%исследователь данных%'
  OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
  OR lower(v.name) LIKE '%machine learning%'
  OR lower(v.name) LIKE '%машинн%обучен%'
        )
  AND (
    lower(v.key_skills) LIKE '%python%'
	  )	   
'''	

query_3_20_extra_df= pd.read_sql_query(query_3_20_extra, connection)

print('Процент вакансий для дата-сайентистов где ключевой навык - Python, от общего числа вакансий для дата-сайентистов равен '
      + str(round(100 * int(query_3_20_extra_df[query_3_20_extra_df['skills']=='DS_Python']['ds_vacancy_num'].iloc[0])
                  /int(query_3_20_extra_df[query_3_20_extra_df['skills']=='DS_vacancy']['ds_vacancy_num'].iloc[0]),1))+'%.')  

Процент вакансий для дата-сайентистов где ключевой навык - Python, от общего числа вакансий для дата-сайентистов равен 73.1%.


* Приложение-6

In [148]:
query_3_22_df['salary'] = query_3_22_df['salary'].apply(lambda x: round(x,0))
display("Распределение средней зарплаты от опыта работы:", query_3_22_df)

'Распределение средней зарплаты от опыта работы:'

,experience,salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


* Приложение-7

In [149]:
query_3_5_extra = f'''
SELECT 
a.name  area,
AVG(COALESCE((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to, 0)) salary
  FROM vacancies v
  JOIN areas a ON v.area_id = a.id
  WHERE v.salary_from IS NOT null
  OR v.salary_to IS NOT NULL
  GROUP BY a.name 
ORDER BY  salary DESC
LIMIT 5
'''
query_3_5_extra_extra_df= pd.read_sql_query(query_3_5_extra, connection)
print('Пятёрка регионов лидеров по размеру средней зарплаты:\n',  query_3_5_extra_extra_df)

Пятёрка регионов лидеров по размеру средней зарплаты:
          area     salary
0    Германия  452751.00
1    Болгария  324348.00
2     Испания  300000.00
3  Черногория  288589.25
4       Литва  263018.50


* Приложение-8

In [150]:
query_3_4_extra = f'''
SELECT 
i.name  industry,
AVG(COALESCE((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to, 0)) salary
  FROM vacancies v
  JOIN employers_industries e_i ON e_i.employer_id = v.employer_id
  JOIN industries i ON i.id = e_i.industry_id
  WHERE v.salary_from IS NOT null
  OR v.salary_to IS NOT NULL
  GROUP BY i.id 
ORDER BY  salary DESC
'''

query_3_4_extra_df= pd.read_sql_query(query_3_4_extra, connection)
query_3_4_extra_df['salary'] = query_3_4_extra_df['salary'].apply(lambda x: round(x,0))
print('Топ пять сфер деятельности по размеру средней зарплаты:\n',  query_3_4_extra_df.iloc[query_3_4_extra_df['salary'].nlargest(5).index])
print('\n')
print('Топ пять сфер деятельности по наименьшей средней зарплаты:\n',  query_3_4_extra_df.iloc[query_3_4_extra_df['salary'].nsmallest(5).index])

Топ пять сфер деятельности по размеру средней зарплаты:
                                     industry    salary
0  Агрохимия (продвижение, оптовая торговля)  381429.0
1                   Агрохимия (производство)  339875.0
2                             Игорный бизнес  191905.0
3          Корма для животных (производство)  168300.0
4                                 Зоомагазин  167613.0


Топ пять сфер деятельности по наименьшей средней зарплаты:
                                 industry   salary
290       Детское питание (производство)  32868.0
289                     Переработка газа  34068.0
288  Архив, библиотека, искусствоведение  36858.0
287                    Ритуальные услуги  37401.0
286                  Фото- и видеоуслуги  40961.0


* Приложение-9

In [151]:
query_3_11_extra = f'''
SELECT 
e.name  employer,
AVG(COALESCE((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to, 0)) salary
  FROM vacancies v
  JOIN employers e ON e.id = v.employer_id
  WHERE v.salary_from IS NOT null
  OR v.salary_to IS NOT NULL
  GROUP BY e.id
ORDER BY  salary DESC
'''

query_3_11_extra_df= pd.read_sql_query(query_3_11_extra, connection)
query_3_11_extra_df['salary'] = query_3_11_extra_df['salary'].apply(lambda x: round(x,0))
print('Топ работодателей по размеру средней зарплаты:\n',  query_3_11_extra_df.iloc[query_3_11_extra_df['salary'].nlargest(5).index])

Топ работодателей по размеру средней зарплаты:
           employer     salary
0  Alpha Personnel  1000000.0
1              ЦТП   650000.0
2           RuWork   555261.0
3    Петухова Алия   552700.0
4      BeRecruited   550000.0


* Приложение-10

In [152]:
query_3_12_extra = f'''
SELECT
e.name employer_name,
COUNT(DISTINCT v.id) num_vacancies
  FROM vacancies v
  JOIN employers e ON e.id = v.employer_id
  WHERE employer_id IS NOT NULL and area_id IS NOT NULL
  GROUP BY e.id
  ORDER BY num_vacancies DESC
  LIMIT 5
'''
query_3_12_extra_df= pd.read_sql_query(query_3_12_extra, connection)
print('Топ работодателей по количеству вакансий:\n',  query_3_12_extra_df)

Топ работодателей по количеству вакансий:
    employer_name  num_vacancies
0         Яндекс           1933
1     Ростелеком            491
2       Тинькофф            444
3           СБЕР            428
4  Газпром нефть            331


* Приложение-11

In [153]:
query_3_41_extra = f'''
SELECT 
i.name  industry,
COUNT(DISTINCT v.id) num_vacancies
  FROM vacancies v
  JOIN employers_industries e_i ON e_i.employer_id = v.employer_id
  JOIN industries i ON i.id = e_i.industry_id
  GROUP BY i.id 
ORDER BY  num_vacancies DESC
LIMIT 5
'''
query_3_41_extra_df= pd.read_sql_query(query_3_41_extra, connection)
print('Топ 5 сфер деятельности по количеству вакансий:\n',  query_3_41_extra_df)

Топ 5 сфер деятельности по количеству вакансий:
                                             industry  num_vacancies
0                Разработка программного обеспечения          12499
1  Системная интеграция,  автоматизации технологи...          11034
2  Интернет-компания (поисковики, платежные систе...           6413
3                                               Банк           2742
4                                    Мобильная связь           1585
